# Imports

In [1]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
import unicodedata

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [3]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [4]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [5]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [6]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [7]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['Ah, no. Vos no te vas. Vos arrojaste la piedra. Ahora no escondas la mano.', '¿Pero cómo podés inventar una cosa así? ¿Con quién? ¿Con quién?', 'Hola, hola, sí, soy yo. ¿Dónde?', 'Sí, lo dijiste.', 'Ah... Señor...', 'No. Ya te dije que a mí no me gusta hacer eso.', 'Dice que ya hirvió los ravioles, pero se le consumió el agua y tiene demasiada harina.', 'Vos sí que tenés ojo clínico.', 'Realmente no vale la pena.', '¿Qué hacés vos para no sentir remordimiento?']
✅ Se extrajeron 1000 frases de Relatos Salvajes.


### Fuente 6 (beta): Mixture de oraciones

In [8]:
cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)


['Los móviles de LG tenían diseños únicos. ¿Cómo confesó el chico según la autoridad judicial que ha revisado el proceso?',
 'La sede de la FIFA está ubicada en la ciudad de Zúrich. ¿Cuál es la universidad peruana con mayor trascendencia a nivel estatal?',
 '¿Qué es lo primero que inventó Pep Torres? Las tarjetas de memoria SanDisk son fiables.',
 '¿Cómo está el desempleo en el país español? El reloj Casio G-Shock es muy resistente.',
 'La investigación del CSIC reveló nuevos datos sobre el universo. ¿Cómo son los números de incidencia del cáncer de pulmón entre mujeres?']

## Juntamos las fuentes

In [9]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['¿Qué mide este estudio anual realizado por Reporteros Sin Fronteras?',
 '¿Cuánto tarda el láser en grabar la letra que falta? Machu Picchu en Perú es una antigua ciudadela inca.',
 '¿Cuántos feminicidios diarios ocurren en México según la ONU? ¿Qué está prohibido para las empresas de EEUU en Cuba?',
 'Alienware es la marca de gaming de Dell.',
 '¿Cuál fue la posición de Fauré entre sus hermanos?']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [11]:
from transformers import BertTokenizerFast, BertModel
import torch

model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [11]:
import re
from collections import defaultdict
from utils.datautils import _get_capitalization_type

def tiene_acento(word):
    word = re.sub(r"ñ", "n", word).lower()
    word = re.sub(r"#", "", word)
    cleaned_word = word.encode("ascii", "ignore").decode("utf-8")
    return word != cleaned_word

PUNCT_TAGS = {"Ø": 0, ",": 1, ".": 2, "?": 3, "¿": 4}

def procesar_oracion(sentence: str, tokenizer):
    # Extraemos palabras con posibles signos de puntuación adyacentes
    matches = list(re.finditer(r"\b(\w|' )+[^\s\w]?\b", sentence, flags=re.UNICODE))
    original_words = [m.group(0) for m in matches]
    # Limpiamos para el tokenizador (solo letras)
    cleaned_words = [re.sub(r"[^A-Za-zÀ-ÿ]", "", w).lower() for w in original_words]

    # Tokenizamos manteniendo la correspondencia word_ids
    encoding = tokenizer(
        cleaned_words,
        is_split_into_words=True,
        return_attention_mask=True,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0])
    word_ids = encoding.word_ids(batch_index=0)

    # Agrupamos índices de subtokens por palabra
    word_to_token_idxs = defaultdict(list)
    for idx, wid in enumerate(word_ids):
        if wid is not None:
            word_to_token_idxs[wid].append(idx)

    total = len(cleaned_words)
    output = []
    for idx, wid in enumerate(word_ids):
        if wid is None:
            continue

        token = tokens[idx]
        prev_tok = tokens[idx-1] if idx > 0 else None
        next_tok = tokens[idx+1] if idx < len(tokens)-1 else None

        # Identificamos primer y último subtoken de la palabra
        is_first = idx == word_to_token_idxs[wid][0]
        is_last = idx == word_to_token_idxs[wid][-1]

        # Detección de acento en la palabra original
        has_accent = int(any(c in original_words[wid] for c in "áéíóúÁÉÍÓÚñÑüÜ"))
        # Posición normalizada de la palabra en la oración
        pos_norm = round(wid / (total - 1), 2) if total > 1 else 0.0

        lead_char = sentence[matches[wid].start() - 1] if matches[wid].start() > 0 else 'Ø'
        start_punc = PUNCT_TAGS.get(lead_char, 0) if is_first else 0

        trail_char = sentence[matches[wid].end()] if matches[wid].end() < len(sentence) else 'Ø'
        end_punc = PUNCT_TAGS.get(trail_char, 0) if is_last else 0
        # Capitalization type
        cap_type = _get_capitalization_type(original_words[wid].strip("¿?.,"))

        output.append({
            "word": cleaned_words[wid],
            "token": tokenizer.convert_tokens_to_ids(token),
            "prev_token": tokenizer.convert_tokens_to_ids(prev_tok) if prev_tok and prev_tok != '[CLS]' else -1,
            "next_token": tokenizer.convert_tokens_to_ids(next_tok) if next_tok and next_tok != '[SEP]' else -1,
            "has_accent": has_accent,
            "position": pos_norm,
            "starting_punctuation_type": start_punc,
            "ending_punctuation_type": end_punc,
            "capitalization_type": cap_type,
        })

    return output

procesar_oracion("¿y vos ¿como?", tokenizer)

[{'word': 'y',
  'token': 193,
  'prev_token': -1,
  'next_token': 63299,
  'has_accent': 0,
  'position': 0.0,
  'starting_punctuation_type': 4,
  'ending_punctuation_type': 0,
  'capitalization_type': 0},
 {'word': 'vos',
  'token': 63299,
  'prev_token': 193,
  'next_token': 10225,
  'has_accent': 0,
  'position': 0.5,
  'starting_punctuation_type': 0,
  'ending_punctuation_type': 0,
  'capitalization_type': 0},
 {'word': 'como',
  'token': 10225,
  'prev_token': 63299,
  'next_token': -1,
  'has_accent': 0,
  'position': 1.0,
  'starting_punctuation_type': 4,
  'ending_punctuation_type': 3,
  'capitalization_type': 0}]

In [12]:
dataset = []
count = 0
for sentence in oraciones_raw:
    if count % 50 == 0:
      print(f"vamos {count}")
    dataset.append(procesar_oracion(sentence, tokenizer))
    count += 1

flattened_dataset = [item for sublist in dataset for item in sublist]

vamos 0
vamos 50
vamos 100
vamos 150
vamos 200
vamos 250
vamos 300
vamos 350
vamos 400
vamos 450
vamos 500
vamos 550
vamos 600
vamos 650
vamos 700
vamos 750
vamos 800
vamos 850
vamos 900
vamos 950
vamos 1000
vamos 1050
vamos 1100
vamos 1150
vamos 1200
vamos 1250
vamos 1300
vamos 1350
vamos 1400
vamos 1450
vamos 1500
vamos 1550
vamos 1600
vamos 1650
vamos 1700
vamos 1750
vamos 1800
vamos 1850
vamos 1900
vamos 1950
vamos 2000
vamos 2050
vamos 2100
vamos 2150
vamos 2200
vamos 2250
vamos 2300
vamos 2350
vamos 2400
vamos 2450
vamos 2500
vamos 2550
vamos 2600
vamos 2650
vamos 2700
vamos 2750
vamos 2800
vamos 2850
vamos 2900
vamos 2950
vamos 3000
vamos 3050
vamos 3100
vamos 3150
vamos 3200
vamos 3250
vamos 3300
vamos 3350
vamos 3400
vamos 3450
vamos 3500
vamos 3550
vamos 3600
vamos 3650
vamos 3700
vamos 3750
vamos 3800
vamos 3850
vamos 3900
vamos 3950
vamos 4000
vamos 4050
vamos 4100
vamos 4150
vamos 4200
vamos 4250
vamos 4300
vamos 4350
vamos 4400
vamos 4450
vamos 4500
vamos 4550
vamos 4600


In [62]:
# item['prev_token'], item['next_token'],
X = [[item['token'],  item['has_accent'], item['position']] for item in flattened_dataset]
y_capitalization = [item['capitalization_type'] for item in flattened_dataset]
y_ending_punctuation = [item['ending_punctuation_type'] for item in flattened_dataset]
y_starting_punctuation = [item['starting_punctuation_type'] for item in flattened_dataset]

print("Distribución de clases en y_capitalization:")
print(Counter(y_capitalization))

print("Distribución de clases en y_ending_punctuation:")
print(Counter(y_ending_punctuation))

print("Distribución de clases en y_starting_punctuation:")
print(Counter(y_starting_punctuation))


Distribución de clases en y_capitalization:
Counter({0: 306455, 1: 96921, 3: 4417, 2: 1560})
Distribución de clases en y_ending_punctuation:
Counter({0: 374049, 2: 13627, 3: 11253, 1: 10424})
Distribución de clases en y_starting_punctuation:
Counter({0: 397536, 4: 11729, 2: 82, 3: 3, 1: 3})


In [63]:
import random
from collections import defaultdict

def undersample(
    X,
    y,
    freq,
):
    # 1) Agrupo índices por clase
    idx_by_class = defaultdict(list)
    for i, label in enumerate(y):
        idx_by_class[label].append(i)

    selected_idxs = []
    rnd = random.Random(42)
    for cls, indices in idx_by_class.items():
        if cls in freq:
            desired = freq[cls]
            available = len(indices)
            if desired > available:
                desired = available
            chosen = rnd.sample(indices, desired)
        else:
            # Si la clase no está en freq, incluyo todas las muestras
            chosen = list(indices)
        selected_idxs.extend(chosen)

    # 3) Barajo y reconstruyo X_res e y_res
    rnd.shuffle(selected_idxs)
    X_res = [X[i] for i in selected_idxs]
    y_res = [y[i] for i in selected_idxs]
    return X_res, y_res


In [64]:
undersample_freq = {
    0: 150000,
    1: 40000,
}
X_capitalization, y_capitalization = undersample(X, y_capitalization,
                                                  undersample_freq)

undersample_freq_punctuation = {
    0: 60000,
}
X_ending_punctuation, y_ending_punctuation = undersample(X, y_ending_punctuation,
                                            undersample_freq_punctuation)

undersample_freq_punctuation = {
    0: 100000,
}
X_starting_punctuation, y_starting_punctuation = undersample(X, y_starting_punctuation,
                                            undersample_freq_punctuation)

In [16]:
# ARBOL PARA CAPITALIZACION
from sklearn.ensemble import RandomForestClassifier

X_train_cap, X_test_cap, y_train_cap, y_test_cap = train_test_split(X_capitalization, y_capitalization, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(list(X_train_cap), list(y_train_cap))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [65]:
# ARBOL PARA PUNTUACION INICIAL

X_train_punc_start, X_test_punc_start, y_train_punc_start, y_test_punc_start = train_test_split(X_starting_punctuation, y_starting_punctuation, test_size=0.2, random_state=42)
clf_punc_start = RandomForestClassifier(n_estimators=100, random_state=42)

clf_punc_start.fit(list(X_train_punc_start), list(y_train_punc_start))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [66]:
# ARBOL PARA PUNTUACION FINAL

X_train_punc_end, X_test_punc_end, y_train_punc_end, y_test_punc_end = train_test_split(X_ending_punctuation, y_ending_punctuation, test_size=0.2, random_state=42)
clf_punc_end = RandomForestClassifier(n_estimators=100, random_state=42)

clf_punc_end.fit(list(X_train_punc_end), list(y_train_punc_end))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [67]:
score = clf_punc_end.score(X_test_punc_end, y_test_punc_end)
print(f"Accuracy end: {score}")

score = clf_punc_start.score(X_test_punc_end, y_test_punc_end)
print(f"Accuracy start: {score}")

Accuracy end: 0.6845391112743298
Accuracy start: 0.6068936572058129


In [ ]:
import random
import re
from typing import Any, Dict, List
from collections import Counter
from joblib import load
import pandas as pd

ACCENT_RE = re.compile(r"[áéíóúÁÉÍÓÚñÑüÜ]")
DEFAULT_PUNCT_MAP: Dict[int, str] = {0: "", 1: ",", 2: ".", 3: "?", 4: "¿"}
brands = {"mcdonald's": "McDonald's",}

def random_forest_predict_and_reconstruct_2(
    cap_model: Any,
    punct_start_model: Any,
    punct_end_model: Any,
    sentence: str,
    tokenizer: Any,
    punct_map: Dict[int, str] = None,
    verbose: bool = False
) -> pd.DataFrame:
    if punct_map is None:
        punct_map = DEFAULT_PUNCT_MAP

    flat_toks: List[str] = tokenizer.tokenize(sentence)
    total = len(flat_toks)
    denom = total - 1 if total > 1 else 1
    ptr = 0
    rows: List[Dict[str, Any]] = []
    instance_id = 1
    new_words: List[str] = []
    for word in sentence.split():
        subtoks = tokenizer.tokenize(word)
        caps_preds: List[int] = []
        start_pred = 0
        end_pred = 0

        for i in range(len(subtoks)):
            tok = flat_toks[ptr]
            if verbose:
                print("Token:", tok)
            tok_id = tokenizer.convert_tokens_to_ids(tok)
            has_accent = 1 if ACCENT_RE.search(tok) else 0
            norm_pos = ptr / denom
            feats = [tok_id, has_accent, norm_pos]
            caps_preds.append(cap_model.predict([feats])[0])
            if i == 0:
                start_pred = punct_start_model.predict([feats])[0]
            if i == len(subtoks) - 1:
                end_pred = punct_end_model.predict([feats])[0]
            ptr += 1

        counter = Counter(caps_preds)
        if caps_preds[0] == 1:
            cap_choice = 1
        elif counter.get(2, 0) > 1:
            cap_choice = 2
        else:
            cap_choice = counter.most_common(1)[0][0]

        if cap_choice == 1:
            mod = word.capitalize()
        elif cap_choice == 2:
            if word.lower() in brands:
                mod = brands[word.lower()]
            else:
                mod = "".join(
                    c.upper() if random.random() > 0.5 else c.lower()
                    for c in word
                )
        elif cap_choice == 3:
            mod = word.upper()
        else:
            mod = word

        prefix = punct_map.get(start_pred, "")
        suffix = punct_map.get(end_pred,   "")
        mod = f"{prefix}{mod}{suffix}"

        new_words.append(mod)

        for idx, tok in enumerate(subtoks):
            token_id = tokenizer.convert_tokens_to_ids(flat_toks[ptr - len(subtoks) + idx])
            rows.append({
                "instancia_id": instance_id,
                "token_id": token_id,
                "token": tok,
                "punt_inicial": start_pred if idx == 0 else 0,
                "punt_final": end_pred if idx == len(subtoks) - 1 else 0,
                "capitalización": cap_choice
            })
        instance_id += 1

    df = pd.DataFrame(rows)
    return df, " ".join(new_words)

entrada = ""
df, salida = random_forest_predict_and_reconstruct_2(
        clf,
        clf_punc_start,
        clf_punc_end,
        entrada,
        tokenizer,
        verbose=False
)

print(df)
print(salida)

   instancia_id  token_id     token  punt_inicial  punt_final  capitalización
0             1     11419        ex             0           0               1
1             1     55852      ##pl             0           0               1
2             1     10545       ##í             0           0               1
3             1     67354     ##cam             0           0               1
4             1     10112       ##e             0           1               1
5             2     10125        el             0           0               0
6             3     35700    método             0           3               1
7             4     10104        de             0           0               0
8             5     10109        la             0           0               0
9             6     54178  potencia             0           2               1
Explícame, el Método? de la Potencia.


## Export modelo

In [14]:
torch.save(model.state_dict(), "model_bidirec_attention.pt")